# NOMBRE COMPLETO: DANIEL FELIPE QUINTERO JIMENEZ

# Preguntas Teóricas
## 1. ¿Cuáles son las diferencias fundamentales entre los modelos encoder-only, decoder-only y encoder-decoder en el contexto de los chatbots conversacionales? Explique qué tipo de modelo sería más adecuado para cada caso de uso y por qué.

| Tipo de Modelo     | Características Principales                                                                 | Casos de Uso en Chatbots                                      | Ejemplos de Modelos     |
|--------------------|---------------------------------------------------------------------------------------------|---------------------------------------------------------------|--------------------------|
| **Encoder-only**   | Solo codifica texto de entrada. Bidireccional. No genera texto.                            | Clasificación de intenciones, análisis de sentimientos        | BERT, RoBERTa            |
| **Decoder-only**   | Genera texto palabra por palabra. Autoregresivo. Unidireccional.                            | Generación de respuestas, completado de texto                 | GPT, GPT-2, GPT-3        |
| **Encoder-Decoder**| Codifica la entrada y luego genera una salida basada en esa codificación.                  | Traducción, resumen, generación controlada de respuestas      | T5, BART, mT5            |

### ¿Qué tipo de modelo sería más adecuado para cada caso de uso y por qué?

- **Encoder-only**: Útil para comprender lo que dice el usuario, como en la clasificación de intenciones o detección de entidades. No sirve para generar respuestas por sí solo.
- **Decoder-only**: Ideal para generar texto fluido y coherente, como respuestas en un chatbot. Usado cuando la prioridad es la generación del lenguaje.
- **Encoder-Decoder**: Útil cuando se necesita transformar texto de una forma a otra, como en chatbots que resumen información, traducen, o responden con base en entradas complejas.

---

## 2. Explique el concepto de "temperatura" en la generación de texto con LLMs. ¿Cómo afecta al comportamiento del chatbot y qué consideraciones debemos tener al ajustar este parámetro para diferentes aplicaciones?

## ¿Qué es la Temperatura en la Generación de Texto con LLMs?

La **temperatura** es un hiperparámetro que se utiliza durante la generación de texto con modelos de lenguaje como GPT para controlar la **aleatoriedad** de las predicciones.

### ¿Cómo Funciona?

Durante la generación, el modelo calcula una distribución de probabilidad sobre el siguiente token (palabra o subpalabra). La temperatura modifica esta distribución:

- Se aplica una fórmula como esta:  
P_i = exp(log(P_i) / T) / sum_j(exp(log(P_j) / T))

Donde `T` es la temperatura.

### Efecto de la Temperatura

- **Temperatura = 1.0** (valor por defecto):  
Comportamiento **estándar** del modelo.

- **Temperatura < 1.0** (por ejemplo, 0.2 a 0.7):  
La distribución se **agudiza** → el modelo es **más conservador y repetitivo**.  
Útil cuando se necesita **respuestas precisas, coherentes y controladas**.

- **Temperatura > 1.0** (por ejemplo, 1.2 o más):  
La distribución se **aplana** → el modelo es **más creativo, impredecible y variado**, pero puede volverse incoherente.  
Útil para tareas como escritura creativa o brainstorming.

### Ejemplos de Aplicación

| Aplicación                     | Temperatura Recomendada | Justificación                                               |
|-------------------------------|--------------------------|-------------------------------------------------------------|
| Chatbot para atención médica  | 0.3 - 0.5                | Respuestas seguras, formales, coherentes                    |
| Asistente educativo            | 0.5 - 0.7                | Cierta flexibilidad, pero con precisión en las explicaciones|
| Generador de historias        | 1.0 - 1.3                | Mayor creatividad y diversidad de narrativas                |
| Chat informal o roleplay      | 0.8 - 1.2                | Conversaciones variadas, espontáneas y menos rígidas        |

### Consideraciones al Ajustar la Temperatura

- No existe un valor único ideal: **depende del contexto de uso**.
- Temperaturas muy altas pueden generar incoherencias o errores.
- Se puede combinar con otros parámetros como `top_k` o `top_p` para refinar aún más el control del texto generado.

---

**Conclusión:**  
La temperatura es clave para modular el comportamiento del chatbot. Ajustarla correctamente puede marcar la diferencia entre una respuesta aburrida y repetitiva o una conversación natural y efectiva.




## 3. Describa las técnicas principales para reducir el problema de "alucinaciones" en chatbots basados en LLMs. ¿Qué estrategias podemos implementar a nivel de inferencia y a nivel de prompt engineering para mejorar la precisión factual de las respuestas?

## Reducción de Alucinaciones en Chatbots Basados en LLMs

Las **alucinaciones** son respuestas generadas por modelos de lenguaje que parecen correctas pero que son **fácticamente incorrectas o inventadas**. Este es uno de los desafíos principales en el uso de LLMs en aplicaciones críticas como educación, medicina o derecho.

---

### Técnicas para Reducir Alucinaciones

#### 🧠 A. A Nivel de Inferencia

1. **Temperatura baja**
   - Usar valores de temperatura entre `0.2` y `0.5` reduce la creatividad del modelo, promoviendo respuestas más seguras y precisas.

2. **Restricción de tokens (top-k / top-p sampling)**
   - Limitar la selección de tokens a los más probables (`top_k`) o acumulando una probabilidad límite (`top_p`) ayuda a evitar resultados improbables o aleatorios.

3. **Consulta iterativa y verificación**
   - Generar respuestas en múltiples pasos o pedir al modelo que **verifique su respuesta** antes de responder definitivamente.

4. **Recuperación basada en documentos (RAG)**
   - Combinar el LLM con un sistema de recuperación de documentos externos (como una base de datos o motor de búsqueda) para responder con información verificada.

---

#### ✍️ B. A Nivel de Prompt Engineering

1. **Instrucciones claras y específicas**
   - Instrucciones como _“Responde solo si estás seguro”_ o _“Si no sabes la respuesta, responde ‘No lo sé’”_ pueden reducir invenciones.

2. **Few-shot prompting con ejemplos reales**
   - Incluir ejemplos correctos y bien estructurados en el prompt para establecer un patrón deseado.

3. **Uso de cadenas de pensamiento ("chain-of-thought")**
   - Pedir al modelo que **razone paso a paso** puede mejorar la precisión lógica y reducir errores fácticos.

4. **Reforzar con roles o contexto de confiabilidad**
   - Frases como _“Eres un experto en medicina. Responde con hechos basados en evidencia científica”_ pueden guiar al modelo hacia mayor rigor.

5. **Incorporación de fuentes o referencias**
   - Pedir al modelo que cite una fuente o diga de dónde proviene la información:  
     _“Incluye la fuente de cada dato que menciones”_.

---

### Ejemplo de Prompt Mejorado

```plaintext
Eres un asistente legal que solo responde con información verificada. Si no tienes certeza, responde: "No tengo suficiente información para responder con precisión".

Pregunta: ¿Cuál es la edad mínima para votar en Argentina?

Conclusión
Reducir las alucinaciones en LLMs requiere una combinación de ajustes técnicos e ingeniería de prompts. No existe una solución única, pero aplicar varias de estas estrategias en conjunto puede mejorar considerablemente la precisión factual del chatbot.


